In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [3]:
# email account details
sender_email = 'stojanovic.jana@outlook.com'

receiver_email = 'jana.stojanovic@bfe.admin.ch'
subject = 'Test Email'
body = 'Test sending an email from Python using Outlook.'

In [4]:
# create message object
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = subject

# attach body to message
message.attach(MIMEText(body, 'plain'))

In [6]:
# connect to Outlook's SMTP server
smtp_server = 'smtp.office365.com'
smtp_port = 587